In [1]:
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
import torch.nn as nn
from tqdm import tqdm 
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt


/home/mmc/anaconda3/envs/CPR_YOLO/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "3"

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'


train_df = pd.read_csv('/home/mmc/disk2/duck/cap/cnn/drink_2/RMBG_640/train/drink.csv')
val_df = pd.read_csv('/home/mmc/disk2/duck/cap/cnn/drink_2/RMBG_640/val/drink.csv')
num_classes = 10


print(f'train shape: {train_df.shape}\nval shape: {val_df.shape}')

In [ ]:
class PixelDropout(object):
    def __init__(self, dropout_prob=0.1):
        self.dropout_prob = dropout_prob

    def __call__(self, img):
        mask = torch.rand_like(img) > self.dropout_prob
        img = img * mask
        return img


class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, transform=None):
        self.img_labels = annotations_file
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_labels.iloc[idx]['path']
        image = Image.open(img_path).convert('RGB')
        label = self.img_labels.iloc[idx]['class']
        if self.transform:
            image = self.transform(image)
        return image, label


# 데이터 전처리
transform = transforms.Compose([
    # transforms.Resize((224, 224)),
    # transforms.RandomApply([transforms.RandomResizedCrop((224, 224))], p=0.3),
    # transforms.RandomHorizontalFlip(),  # 좌우 반전
    # transforms.RandomRotation(10),  # 30도 이내의 무작위 회전
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 밝기, 대비, 채도, 색조 변화
    transforms.ToTensor(),
    # PixelDropout(dropout_prob=0.1),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 사용자 정의 데이터셋 인스턴스 생성
train_dataset = CustomImageDataset(train_df,transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = CustomImageDataset(val_df, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)


In [ ]:
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomImageClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CustomImageClassifier, self).__init__()
        self.base_model = timm.create_model('resnet50.a1_in1k', pretrained=True, num_classes=num_classes)

    def forward(self, x):
        x = self.base_model(x)
        x = F.softmax(x, dim=1)  # 소프트맥스 적용
        return x


# 모델을 생성합니다.
model = CustomImageClassifier(num_classes)

# 저장된 가중치를 로드합니다.
# model.load_state_dict(torch.load('./snack_pt_0518_resnet18/last_0.033.pt'))

# 모델을 적절한 장치로 이동시킵니다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.1, verbose=True)


In [ ]:
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

# 초기 최소 검증 손실값 설정
min_val_loss = float('inf')

# 학습 및 검증 과정
train_losses = []
val_losses = []

for epoch in range(100): 
    model.train()
    train_loss, val_loss = 0.0, 0.0
    
    # 학습 부분
    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # 검증 부분
    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    # 에폭별 손실 출력
    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}')

    # 스케줄러 업데이트
    scheduler.step(avg_val_loss)

    # 검증 손실이 개선되었는지 확인하고 모델 저장
    if avg_val_loss < min_val_loss:
        min_val_loss = avg_val_loss
        torch.save(model.state_dict(), './kang/snack_pt/best.pt')
        print(f"Model saved: Epoch {epoch+1} with Val Loss: {avg_val_loss:.4f}")

    # # 5 에폭마다 손실값 시각화
    # if (epoch + 1) % 5 == 0:
        
    #     plt.figure(figsize=(10, 5))
    #     plt.plot(range(1, epoch+2), train_losses, label='Train Loss')
    #     plt.plot(range(1, epoch+2), val_losses, label='Validation Loss')
    #     plt.xlabel('Epoch')
    #     plt.ylabel('Loss')
    #     plt.title('Train and Validation Loss')
    #     plt.legend()
    #     plt.grid(True)
    #     plt.savefig(f'./snack_pt_0520_resnet18/loss_plot_epoch_{epoch+1}.png')
    #     plt.show()

# 마지막 모델 상태 저장
torch.save(model.state_dict(), f'./snack_pt_0520_resnet18/last_{min_val_loss:.2f}.pt')
